# Python для анализа данных

## Работа с форматами XML и JSON


# XML

До этого мы с вами собирали данные вручную, обращаясь к html страницам, размеченным для отображения в браузере. Но данные также можно собирать и через API -  — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером. 

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

```xml
<group>
    <number>134</number>
    <student>
        <firstname>Виталий</firstname>
        <lastname>Иванов</lastname>
    </student>
    <student>
        <firstname>Мария</firstname>
        <lastname>Петрова</lastname>
    </student>
</group>
```

Для обработки XML-файлов можно использовать тот же пакет *Beautiful Soup*, который мы уже использовали для работы с HTML. Единственное различие — нужно указать дополнительный параметр `feautres="xml"` при вызове функции `BeautifulSoup` — чтобы он не искал в документе HTML-теги.

In [26]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [2]:
!pip install lxml

In [27]:
from bs4 import BeautifulSoup

obj = BeautifulSoup(group, features="lxml")
print(obj.prettify())

<html>
 <body>
  <group>
   <number>
    134
   </number>
   <student>
    <firstname>
     Виталий
    </firstname>
    <lastname>
     Иванов
    </lastname>
   </student>
   <student>
    <firstname>
     Мария
    </firstname>
    <lastname>
     Петрова
    </lastname>
   </student>
  </group>
 </body>
</html>


Номер группы можно найти, например, вот так - для каждого объекта через точку указываем его атрибут, в который надо спуститься. 

In [28]:
obj.group.number.text # последний атрибут текст, точно также как делали в html

'134'

Но это работает только тогда, когда тэг уникальный. В других случаях, парсер всегда будет попадать в первый child-тэг, который он встретил по пути вниз.

In [29]:
obj.group.student.lastname.text # до Петровой так не добраться

'Иванов'

Перечислить всех студентов можно с помощью цикла (похожая структура у нас была и в обработке html).

In [6]:
for student in obj.group.find_all('student'):
    print(student.lastname.text, student.firstname.text)

Иванов Виталий
Петрова Мария


По сути, главное отличие xml от html, что работать вы будете не со стандартизированными структурами. Поэтому перед работой придется поиграть в детективов - запросить данные и внимательно изучить расположение узлов, чтобы понять, какие тэги вас интересуют.

XML легко представить в виде дерева, где есть главный узел (parent) и его "дети".

![](https://www.py4e.com/images/xml-tree.svg)  
*Источник: Python for Everybody, C.Severance*

Кроме BS парсить xml можно и с помощью других библиотек. Например, ElementTree.


In [30]:
import xml.etree.ElementTree as ET

In [31]:
tree = ET.fromstring(group)
list(tree) # посмотрим, что внутри

[<Element 'number' at 0x7fc8c81bd630>,
 <Element 'student' at 0x7fc8c81bd5e0>,
 <Element 'student' at 0x7fc8c81bd450>]

Cинтаксис очень похож на BS. Добрались до первой фамилии.

In [32]:
tree.find('student').find('lastname').text

'Иванов'

In [33]:
for element in tree.findall('student'):
    print(element)
    print(element.find('lastname').text)

<Element 'student' at 0x7fc8c81bd5e0>
Иванов
<Element 'student' at 0x7fc8c81bd450>
Петрова


Можно немного упростить код, включив дочерний тэг в findall.

In [34]:
for element in tree.findall('student/lastname'):
    print(element.text)

Иванов
Петрова


# Задача
По ссылке данные в формате xml.
http://py4e-data.dr-chuck.net/comments_42.xml

Посчитайте все комментарии в этом документе (поля count).

In [35]:
import requests
data = requests.get('http://py4e-data.dr-chuck.net/comments_42.xml').text

tree = ET.fromstring(data)

total = 0
for element in tree.findall('comments/comment/count'):
#     print(type(element.text))
    total += int(element.text)
    
print(total)

2553


# JSON

Другой популярный формат, в котором клиент может отдать вам данные - json. JSON расшифровывается как JavaScript Object Notation и изначально возник как подмножество языка JavaScript (пусть вас не вводит в заблуждение название, этот язык ничего не имеет общего с Java), используемое для описания объектов, но впоследствии стал использоваться и в других языках программирования, включая Python. Различные API могут поддерживать либо XML, либо JSON, либо и то, и другое, так что нам полезно научиться работать с обоими типами данных (например, wiki api могла бы выгрузить нам данные и в формате json при соответствующем запросе).

JSON очень похож на описание объекта в Python и смысл квадратных и фигурных скобок такой же. Правда, есть и отличия: например, в Python одинарные и двойные кавычки ничем не отличаются, а в JSON можно использовать только двойные. Мы видим, что полученный нами JSON представляет собой словарь, значения которого — строки или числа, а также списки или словари, значения которых в свою очередь также могут быть строками, числами, списками, словарями и т.д. То есть получается такая довольно сложная структура данных.

В данный момент тот факт, что перед нами сложная структура данных, видим только мы — с точки зрения Python, j.text это просто такая строка. Однако в модуле requests есть метод, позволяющий сразу выдать питоновский объект (словарь или список), если результат запроса возвращён в формате JSON. Так что нам не придётся использовать никакие дополнительные библиотеки.

Преимущества JSON в том, что мы получаем готовый объект Python и нет необходимости использовать какие-то дополнительные библиотеки для того, чтобы с ним работать. Недостатком является то же самое: зачастую поиск информации в XML-файле может проводиться более эффективно, чем в JSON. 

## JSON (парсинг VK)

Как уже говорилось выше, не все API открытые. Так, чтобы достать информацию из vk вам придется сгенерировать токен с помощью вашего аккаунта. API VK отдает данные в json. Структура тут будет посложнее, чем то, что мы уже видели, поэтому давайте еще потренируемся.

Теперь научимся еще и загружать JSON файл с диска. Для этого нам понадобится модуль json.

In [41]:
import json

with open('vk.json',  'r', encoding='Utf-8') as json_data:
    res_loaded = json.load(json_data) # считываем данные с помощью функции .load()

Здесь у нас выгрузка постов со стены группы ВШЭ.

In [44]:
res_loaded.keys()

dict_keys(['count', 'items'])

In [49]:
type(res_loaded['items'])

list

In [2]:
res_loaded['items'][0]

{'id': 32494,
 'from_id': -132,
 'owner_id': -132,
 'date': 1541422799,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'attachments': [{'type': 'link',
   'link': {'url': 'http://family.hse.ru/event/view/2271',
    'title': 'День карьеры факультета бизнеса и менеджмента',
    'caption': 'family.hse.ru',
    'description': '',
    'photo': {'id': 456239093,
     'album_id': -2,
     'owner_id': 100,
     'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
     'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
     'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
     'width': 150,
     'height': 80,
     'text': '',
     'date': 1541422799}}}],
 'post_source': {'type': 'vk'},
 'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'count': 2, 'user_reposted': 0},
 'views': {'count': 2832}}

In [50]:
res_loaded['items'][1]

{'id': 32493,
 'from_id': -132,
 'owner_id': -132,
 'date': 1541076555,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
 'post_source': {'type': 'vk'},
 'comments': {'count': 0, 'can_post': 1, 'groups_can_post'

In [4]:
res_loaded.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [51]:
res_loaded['items'][50] # первый элемент items - первый пост со всей информацией о нем

{'id': 32099,
 'from_id': -132,
 'owner_id': -132,
 'date': 1531760787,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 1050401,
   'owner_id': -79138567,
   'from_id': -79138567,
   'date': 1531760410,
   'post_type': 'post',
   'text': 'В России есть проблемы с образованием, но часть из них можно решить с помощью грамотного менеджмента.\n\nМагистерская программа НИУ ВШЭ «Экономика и управление образованием» сделает из вас крутого специалиста. Вы научитесь руководить большими образовательными организациями и открывать собственные: школы, курсы, университеты и так далее. \n\nЗадавайте в Прямой линии свои вопросы о поступлении, будущей профессии и образовательном менеджменте:\n\nhttps://thequestion.ru/interview/5663/magistratura-niu-vshe-ekonomika-i-upravlenie-obrazovaniem#',
   'attachments': [{'type': 'doc',
     'doc': {'id': 469433958,
      'owner_id': 27546726,
      'title': 'school3.gif',
      'size': 378724,
      'ext': 'gif',
      'url': 'ht

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [52]:
items = res_loaded['items']
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list
# несколько элементов списка
full_list[0:4]

[[-132, 32494, '', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, '', 1540743239],
 [-132, 32491, '', 1540551399]]

Видно, что в двух первых постах текста не обнаружено, там только картинки, ссылки и репосты.

Из этого списка списков можно легко сделать датафрейм `pandas`.

Но если у ссылок, репостов и картинок нет текста, то наш DataFrame будет выглядет неполноценно. Поэтому добавим условие, что если текст отсутсвует, вставим в DataFrame строку с этой отметкой.

In [53]:
# опять выберем только нужные поля
full_list = []

for item in items:
    if item['text'] == "":
        l = [item['from_id'], item['id'], "Картинка, ссылка или репост", item['date']]
    else:
        l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

In [54]:
full_list

[[-132, 32494, 'Картинка, ссылка или репост', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, 'Картинка, ссылка или репост', 1540743239],
 [-132, 32491, 'Картинка, ссылка или репост', 1540551399],
 [-

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [55]:
import pandas as pd

Создадим датафрейм:

In [56]:
df = pd.DataFrame(full_list, columns = ['From_id', 'Id', 'Text', 'Date_Unix'])
df.head(10)

,From_id,Id,Text,Date_Unix
0,-132,32494,"Картинка, ссылка или репост",1541422799
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555
2,-132,32492,"Картинка, ссылка или репост",1540743239
3,-132,32491,"Картинка, ссылка или репост",1540551399
4,-132,32488,"Картинка, ссылка или репост",1540302306
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523
6,-132,32485,"Картинка, ссылка или репост",1539781575
7,-132,32484,"Картинка, ссылка или репост",1539718883
8,-132,32481,"Картинка, ссылка или репост",1539088885
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях. Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат год-месяц-день-часы-минуты-секунды. Для этого нам понадобится модуль datetime.

In [59]:
from datetime import datetime

In [62]:
def date_norm(date):
    
    """Получает на вход строку данных времени в формате POSIX. 
    Возвращает кортеж (ГГГГ-ММ-ДД, ЧЧ:ММ:СС)"""
    
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    date_norm, time_norm = str_d.split() # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

In [65]:
datetime.fromtimestamp(1539088745).strftime("%Y-%m-%d %H:%M:%S")

'2018-10-09 15:39:05'

In [61]:
?date_norm

Применим нашу функцию к элементам столбца date и создадим новый ‒ `date_norm`.

In [66]:
df['Date_Norm'] = df.Date_Unix.apply(date_norm)

In [67]:
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)"
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)"
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)"
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)"
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)"


Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени ‒ извлекать их из кортежа в date_norm.

In [68]:
df['Date'] = df.Date_Norm.apply(lambda x: x[0])
df['Time'] = df.Date_Norm.apply(lambda x: x[1])
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm,Date,Time
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)",2018-11-05,15:59:59
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)",2018-11-01,15:49:15
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)",2018-10-28,19:13:59
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)",2018-10-26,13:56:39
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)",2018-10-23,16:45:06


In [69]:
del df['Date_Norm']

In [70]:
df.head()

,From_id,Id,Text,Date_Unix,Date,Time
0,-132,32494,"Картинка, ссылка или репост",1541422799,2018-11-05,15:59:59
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,2018-11-01,15:49:15
2,-132,32492,"Картинка, ссылка или репост",1540743239,2018-10-28,19:13:59
3,-132,32491,"Картинка, ссылка или репост",1540551399,2018-10-26,13:56:39
4,-132,32488,"Картинка, ссылка или репост",1540302306,2018-10-23,16:45:06


In [72]:
df[df['Text'] != 'Картинка, ссылка или репост']

,From_id,Id,Text,Date_Unix,Date,Time
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,2018-11-01,15:49:15
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523,2018-10-18,14:52:03
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745,2018-10-09,15:39:05
10,-132,32478,Будущим магистрам! Новая программа факультета ...,1539016393,2018-10-08,19:33:13
11,-132,32476,До окончания регистрации в Осеннюю школу бизне...,1538752645,2018-10-05,18:17:25
13,-132,32474,Продолжается регистрация школьников 9-11 класс...,1538513448,2018-10-02,23:50:48
14,-132,32472,Для абитуриентов Школа логистики факультета би...,1538477565,2018-10-02,13:52:45
16,-132,32470,Приглашаем 29 октября - 4 ноября школьников 9-...,1537815734,2018-09-24,22:02:14
17,-132,32469,"Привет! Ты начинающий дизайнер, программист ил...",1537779781,2018-09-24,12:03:01
21,-132,32462,"Иванова Валерия Станиславовна, документы найде...",1537168381,2018-09-17,10:13:01


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://dev.vk.com/reference).